In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import rioxarray
import rioxarray    
import rasterio
import numpy as np
import pandas as pd
import os
import subprocess as subp
import time
from osgeo import gdal

In [3]:
files = os.listdir('Data/')
files = [elements for elements in files if '.xml' not in elements]
files = [elements for elements in files if '.py' not in elements]
files.sort()

In [4]:
i = files[0]
op_name = i[21:29]

In [5]:
i[:-2]

'SMAP_L2_SM_SP_1AIWDV_20230108T005943_20230108T004628_076E20N_R18290_001.'

In [6]:
file_date = [elements for elements in files if op_name in elements[:29]]


In [7]:
try:
    os.mkdir('Tiff/')
except:
    pass
for i in files:
    xds = rioxarray.open_rasterio("Data/"+i)

    lats = np.array(xds[0]['Soil_Moisture_Retrieval_Data_1km_latitude_1km'])[0].flatten()
    lons = np.array(xds[0]['Soil_Moisture_Retrieval_Data_1km_longitude_1km'])[0].flatten()
    vals = np.array(xds[0]['Soil_Moisture_Retrieval_Data_1km_soil_moisture_1km'])[0].flatten()


    df = pd.DataFrame()
    df['latitude'] = lats
    df['longitude'] = lons
    df['value'] = vals


    da = df.set_index(['latitude', 'longitude']).to_xarray()
    da = da.set_coords(['latitude', 'longitude'])
    da.rio.to_raster('Tiff/'+i[:-3]+'.tif')

    dataset = rasterio.open('Tiff/'+i[:-3]+'.tif', 'r+')
    dataset.driver='GTIFF'
    dataset.dtype=rasterio.float32
    dataset.compress='lzw'
    dataset.nodata=-9999
    dataset.crs='EPSG:4326'
    dataset = None

In [9]:
import geopandas as gpd
gdf = gpd.read_file('../../../../src/data_preprocessing/base_geojson/TL_state_shapefile_for_clip.geojson')

In [25]:
polygon = (((str(gdf['geometry'][0])[16:-3]).replace(",", "")).replace(" ", ", "))[:-38]

In [33]:
files = os.listdir('Tiff/')
files.sort()

In [41]:
files[0][:-23]

'SMAP_L2_SM_SP_1AIWDV_20230108T005943_20230108T004628'